In [ ]:
# Optional: un-comment the line below in case you are not sure if your environment has all the packages installed
#!pip install numpy pandas bs4 lxml grequests --user

In [2]:
# Packages we need in order to perform the scraping
import grequests
from grequests import get
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import os
from datetime import datetime

# EDIT THIS TO YOUR LOCAL FOLDER
#os.chdir('C:\\path\\to\\the\\projects\\folder')

#OPTIONAL: UNCOMMENT IF YOU RUN IT ON GOOGLE COLAB: https://colab.research.google.com/notebooks/intro.ipynb
#os.chdir('/content/')

today = datetime.today().strftime('%m-%d-%Y')

In [ ]:
###################################FUNCTION TO SCRAPE THE DATA FROM THE SEARCH PAGE LINKS
def rentals(alist):
    
    results = pd.DataFrame()
    
    rs = (grequests.get(url) for url in alist)
    responses = grequests.imap(rs, size = 10)

    for response in responses:

        soup = BeautifulSoup(response.text, 'lxml')
        
        div_tag = soup.find_all('div', {'class':'card-list__item'})
        
        for div in div_tag:

            try:
                title = div.find('h2', {'class':'card__title card__title-link'}).text.strip()
            except:
                title = None

            try:
                ttype = div.find('p', {'class':'card__property-amenity card__property-amenity--property-type'}).text.strip()
            except:
                ttype = None

            try:
                bedrooms = div.find('p', {'class':'card__property-amenity card__property-amenity--bedrooms'}).text.strip()
            except:
                bedrooms = None
                
            try:
                bathrooms = div.find('p', {'class':'card__property-amenity--bathrooms'}).text.strip()
            except:
                bathrooms = None

            try:
                area = div.find('p', {'class':'card__property-amenity card__property-amenity--area'}).text.strip()
            except:
                area = None                
                
            try:
                price = div.find('span', {'class':'card__price-value'}).text.strip()
            except:
                price = None
                
            try:
                frequency = div.find('p', {'class':'card__property-amenity--bathrooms'}).text.strip()
            except:
                frequency = None

            try:
                location = div.find('span', {'class':'card__location-text'}).text.strip()
            except:
                location = None                

            try:
                link = div.find('a', {'class':'card card--clickable'})['href']
                link = 'www.propertyfinder.ae' + link
            except:
                link = None
                
            temp_df = pd.DataFrame([[title, ttype, bedrooms, bathrooms, area, price, location, link]], 
                                   columns=['title', 'type', 'bedrooms', 'bathrooms', 'area', 'price', 'location', 'link'])
            results = results.append(temp_df, sort=False).reset_index(drop=True)
    sleep(randint(2,3))
    return results

In [ ]:
#LOAD THE DAILY LIST OF LINKS
pf = pd.read_csv('AllPFPages-' + today + '.csv', sep=',')

#SHOW THE DATAFRAME FOR A QUICK INSPECTION
pf

In [ ]:
#LOAD THE LINKS INTO LIST
links = pf['Links'].tolist()

#PRINT THE TOTAL NUMBER OF PAGES TO SCRAPE
print(len(links))

In [ ]:
#RUN THE SCRAPER AND ADD THE START & END TIME
now = datetime.now()
start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

data = rentals(links)

now = datetime.now()
end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)

In [ ]:
#SAVE THE DATAFRAME TO DISK AS .CSV
data.to_csv('PF_Rentals-' + today + '.csv', sep=',')